In [ ]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import torch.nn.functional as F
from torchsummary import summary

In [ ]:
device = 'cpu'

In [ ]:
print(device)

cpu


In [ ]:
num_epochs =5
batch_size =4
learning_rate = 0.001

In [ ]:
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5))])


In [ ]:
train_dataset = torchvision.datasets.CIFAR10(root='data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.CIFAR10(root='data', train=False, download=True, transform=transform)

Files already downloaded and verified
Files already downloaded and verified


In [ ]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
clases = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

(w-f+2p)/s +1
(32-5+2*p)/s +

In [ ]:
class ConvNet(nn.Module):
      def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 6,3, padding=(1,1))
        self.pool = nn.MaxPool2d(2,2)
        self.conv2 = nn.Conv2d(6, 12,3, padding=(1,1))
        self.conv3 = nn.Conv2d(12, 16,3, padding=(1,1))
        self.drop = nn.Dropout(0.2)
        self.fc1 = nn.Linear(16*4*4,120)
        self.fc2 = nn.Linear(120,84)
        self.fc3 = nn.Linear(84,10)

      def forward(self,x):
        # -> n, 3, 32, 32
        x = self.pool(F.relu(self.conv1(x)))  # -> n, 6, 16, 16
        x = self.pool(F.relu(self.conv2(x)))  # -> n, 12, 8, 8
        x = self.pool(F.relu(self.conv3(x))) #  -> 16,4,4
        x = x.view(-1, 16 * 4 * 4)            # -> n, 400
        x = self.drop(F.relu(self.fc1(x)))               # -> n, 120
        x = self.drop(F.relu(self.fc2(x)))               # -> n, 84
        x = self.fc3(x)                       # -> n, 10
        return x


In [ ]:
model = ConvNet().to(device)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, weight_decay=0.001)


In [ ]:
summary(model, (3, 32, 32))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 6, 32, 32]             168
         MaxPool2d-2            [-1, 6, 16, 16]               0
            Conv2d-3           [-1, 12, 16, 16]             660
         MaxPool2d-4             [-1, 12, 8, 8]               0
            Conv2d-5             [-1, 16, 8, 8]           1,744
         MaxPool2d-6             [-1, 16, 4, 4]               0
            Linear-7                  [-1, 120]          30,840
           Dropout-8                  [-1, 120]               0
            Linear-9                   [-1, 84]          10,164
          Dropout-10                   [-1, 84]               0
           Linear-11                   [-1, 10]             850
Total params: 44,426
Trainable params: 44,426
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.01
Forward/ba

In [ ]:
n_total_steps = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # origin shape: [4, 3, 32, 32] = 4, 3, 1024
        # input_layer: 3 input channels, 6 output channels, 5 kernel size
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 2000 == 0:
            print (f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{n_total_steps}], Loss: {loss.item():.4f}')


print('Finished Training')
PATH = './cnn.pth'
torch.save(model.state_dict(), PATH)


Epoch [1/5], Step [2000/12500], Loss: 2.3084
Epoch [1/5], Step [4000/12500], Loss: 2.2694
Epoch [1/5], Step [6000/12500], Loss: 2.2902
Epoch [1/5], Step [8000/12500], Loss: 2.3102
Epoch [1/5], Step [10000/12500], Loss: 2.2915
Epoch [1/5], Step [12000/12500], Loss: 2.3316
Epoch [2/5], Step [2000/12500], Loss: 2.3209
Epoch [2/5], Step [4000/12500], Loss: 2.1236
Epoch [2/5], Step [6000/12500], Loss: 2.1293
Epoch [2/5], Step [8000/12500], Loss: 2.3456
Epoch [2/5], Step [10000/12500], Loss: 2.5528
Epoch [2/5], Step [12000/12500], Loss: 2.3801
Epoch [3/5], Step [2000/12500], Loss: 1.8929
Epoch [3/5], Step [4000/12500], Loss: 1.7236
Epoch [3/5], Step [6000/12500], Loss: 1.8090
Epoch [3/5], Step [8000/12500], Loss: 1.9257
Epoch [3/5], Step [10000/12500], Loss: 1.7186
Epoch [3/5], Step [12000/12500], Loss: 2.4115
Epoch [4/5], Step [2000/12500], Loss: 1.8875
Epoch [4/5], Step [4000/12500], Loss: 1.6269
Epoch [4/5], Step [6000/12500], Loss: 1.4190
Epoch [4/5], Step [8000/12500], Loss: 1.6605
Epoc

In [ ]:
with torch.no_grad(): # Se deshabilita el siguimiento de los gradientes
    n_correct = 0
    n_samples = 0
    n_class_correct = [0 for i in range(10)] # Se almacena la cantidad de clases correctas
    n_class_samples = [0 for i in range(10)]
    for images, labels in test_loader:
        images = images.to(device) # cargamos a dispositivo
        labels = labels.to(device) # cargamos a dispositivo
        outputs = model(images) # resultadp
        _, predicted = torch.max(outputs, 1) # Calcula las predicciones finales del modelo seleccionando el índice de la clase con el valor de probabilidad más alto.
        n_samples += labels.size(0)
        n_correct += (predicted == labels).sum().item()

        for i in range(batch_size):
            label = labels[i]
            pred = predicted[i]
            if (label == pred):
                n_class_correct[label] += 1
            n_class_samples[label] += 1

    acc = 100.0 * n_correct / n_samples
    print(f'Accuracy of the network: {acc} %')

    for i in range(10):
        acc = 100.0 * n_class_correct[i] / n_class_samples[i]
        print(f'Accuracy of {clases[i]}: {acc} %')

Accuracy of the network: 36.17 %
Accuracy of plane: 39.0 %
Accuracy of car: 60.7 %
Accuracy of bird: 4.1 %
Accuracy of cat: 21.6 %
Accuracy of deer: 23.7 %
Accuracy of dog: 46.0 %
Accuracy of frog: 39.1 %
Accuracy of horse: 48.7 %
Accuracy of ship: 35.7 %
Accuracy of truck: 43.1 %
